# Image retrieval

Match museum images with users' pictures.

Methods: 

 * OpenCV - FLANN Matcher. It's not easy to define a similarity score that is reliable for all the matchings
 * OpenCV - BF Matcher. Less performant than FLANN Matcher
 * Tensorflow - DELF. Returns several false positives but the correct matched image, if available, gives significant results - the number of inliers (matching keypoints) are consistently higher. However there is no magic number for a similarity score.
 * **CNN - Keras**. So far, the best option. Uses a convolutional neural network to extract features.

Others (removed):
 * OpenCV - Contours. Not reliable for images that include details of objects or landscape pictures of rooms.


### Imports

In [2]:
import cv2
import numpy as np
import glob , json
from matplotlib import pyplot as plt

##  FLANN - OpenCV

FLANN applied to images in two folders recursively.

In [3]:
def flann_percent(users_imgs_folder, catalogue_folder):
    """FLANN applied to images included in two folders. 
    Computes a siliarity score based on the proportion of good keypoints and the total numebr of keypoints.
    Returns a JSON file with the best matches.
    Example: flann_percent('images/user_imgs/','images/museum_imgs/')"""

    matches_list = []
    MIN_MATCH_COUNT = 800
    MIN_SIMILARITY_PERCENTAGE = 90
    for catalogue_img in glob.iglob(catalogue_folder+'*'):
        matches_dict = {}
        matches_dict["catalogue_img"] = catalogue_img
        for user_file in glob.iglob(users_imgs_folder+'*'):
            original = cv.imread(user_file,0)
            original = cv.cvtColor(original, cv.COLOR_GRAY2BGR)
            user_image = cv.imread(catalogue_img,0)
            user_image = cv.cvtColor(user_image, cv.COLOR_GRAY2BGR)

            index_params = dict(algorithm=1, trees=5)
            search_params = dict()

            sift = cv.xfeatures2d.SIFT_create()
            kp_1, desc_1 = sift.detectAndCompute(original, None)
            kp_2, desc_2 = sift.detectAndCompute(user_image, None)
            flann = cv.FlannBasedMatcher(index_params, search_params)
            matches = flann.knnMatch(desc_1, desc_2, k=2)
            good_points = []
            for m, n in matches:
                if m.distance > 0.7*n.distance:
                    good_points.append(m)

            number_keypoints = len(kp_1) if len(kp_1) >= len(kp_2) else len(kp_2)
            percentage_similarity = len(good_points) / number_keypoints * 100
            
            print("User: " + user_file)
            print("Catalogue: " + catalogue_img)
            print("Similarity: " + str(int(percentage_similarity)) + "\n")
            
            if percentage_similarity >= MIN_SIMILARITY_PERCENTAGE and len(good_points)>MIN_MATCH_COUNT:
                print("Lenght points:", len(good_points))
                print(len(good_points) , len(kp_1) , len(kp_2))
                matches_dict[user_file] = [percentage_similarity, len(good_points)]
            
            if len(good_points)>MIN_MATCH_COUNT:
                src_pts = np.float32([ kp_1[m.queryIdx].pt for m in good_points ]).reshape(-1,1,2)
                dst_pts = np.float32([ kp_2[m.trainIdx].pt for m in good_points ]).reshape(-1,1,2)
                M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
                matchesMask = mask.ravel().tolist()
                h,w,d = original.shape
                pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
                dst = cv.perspectiveTransform(pts,M)
                img2 = cv.polylines(user_image,[np.int32(dst)],True,255,3, cv.LINE_AA)
                draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
                img3 = cv.drawMatches(original,kp_1,img2,kp_2,good_points,None,**draw_params)
                plt.imshow(img3, 'gray'),plt.show()
                
            else:
                print( "Not enough matches are found - {}/{}".format(len(good_points), MIN_MATCH_COUNT) )
                matchesMask = None
            

    
        matches_list.append(matches_dict)

    with open('images_matching.json', 'w', encoding='utf-8') as outfile:
        json.dump(matches_list, outfile, ensure_ascii=False, indent=1)
    



FLANN applied recursively to images in two folders with Lowe ratio test (to select the good keypoints)

In [4]:
def flann_lowe(users_imgs_folder, catalogue_folder):
    """FLANN applied to images included in two folders. 
    Computes a siliarity score based on the Lowe's ratio.
    Returns: the plot with the best matches.
    Example: flann_lowe('images/user_imgs/','images/museum_imgs/')"""
    for catalogue_img in glob.iglob(catalogue_folder+'*'):
        for user_file in glob.iglob(users_imgs_folder+'*'):
            img1 = cv.imread(catalogue_img, cv.COLOR_BGR2GRAY)  # queryImage
            img2 = cv.imread(user_file, cv.COLOR_BGR2GRAY)  # trainImage


            # Initiate SIFT with FLANN parameters detector
            sift = cv.SIFT_create()

            # find the keypoints and descriptors with SIFT
            kp1, des1 = sift.detectAndCompute(img1,None)
            kp2, des2 = sift.detectAndCompute(img2,None)

            # FLANN parameters
            FLANN_INDEX_KDTREE = 1
            index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
            search_params = dict(checks=50)
            flann = cv.FlannBasedMatcher(index_params,search_params)
            matches = flann.knnMatch(des1,des2,k=2)

            # Need to draw only good matches, so create a mask
            matchesMask = [[0,0] for i in range(len(matches))]

            # ratio test as per Lowe's paper
            good = 0
            for i,(m,n) in enumerate(matches):
                if m.distance < 0.5*n.distance:
                    good +=1
                    matchesMask[i]=[1,0]
            draw_params = dict(matchColor = (0,255,0),
                               singlePointColor = (255,0,0),
                               matchesMask = matchesMask,
                               flags = cv.DrawMatchesFlags_DEFAULT)
            if good> 10:
                im3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)

                match_title= str(good)+ ' good points - kp1:'+(str(len(kp1))) + ' kp2:',(str(len(kp2)))
                print('cat:',catalogue_img,'\nuser:', user_file, '\n',match_title)
                plt.imshow(im3),plt.show()

FLANN applied to two images only.

In [5]:
def flann_(catalogue_img,user_file):
    """FLANN applied to two images. 
    Computes a siliarity score based on the Lowe's ratio.
    Returns: the plot with the best matches.
    Example: flann_('images/im1.jpg','images/im2.jpg')"""
    img1 = cv.imread(catalogue_img, cv.COLOR_BGR2GRAY)  # queryImage
    img2 = cv.imread(user_file, cv.COLOR_BGR2GRAY)  # trainImage


    # Initiate SIFT with FLANN parameters detector
    sift = cv.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)

    # Need to draw only good matches, so create a mask
    matchesMask = [[0,0] for i in range(len(matches))]

    # ratio test as per Lowe's paper
    good = 0
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.75*n.distance:
            good +=1
            matchesMask[i]=[1,0]
    draw_params = dict(matchColor = (0,255,0),
                       singlePointColor = (255,0,0),
                       matchesMask = matchesMask,
                       flags = cv.DrawMatchesFlags_DEFAULT)
    
    im3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)

    match_title= str(good)+ ' good points - kp1:'+(str(len(kp1))) + ' kp2:',(str(len(kp2)))
    print('cat:',catalogue_img,'\nuser:', user_file, '\n',match_title)
    plt.imshow(im3),plt.show()

## Brute-force Matcher - OpenCV

BF applied respectively, to two images only and to images in two folders (recursively).

In [6]:
def bf_(im1, im2):
    """ Brute forse applied on two images. 
    Returns: the plot with all the matches.
    Example: bf_('images/im1.jpg','images/im2.jpg')
    """
    img1 = cv.imread('images/dmh_test/test.jpg', cv.COLOR_BGR2GRAY)  # queryImage
    img2 = cv.imread('images/ig_test/925902_651302864947190_1364658590_n.jpg?_nc_ht=scontent-arn2-1.cdninstagram.com&_nc_cat=102&_nc_ohc=6HwAxf1g7VQAX8tOgkL&tp=1&oh=3bd2381ac6533461890d7b25cd09dc06&oe=603D5C88', cv.COLOR_BGR2GRAY)  # trainImage

    orb = cv.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)

    # Sort them in the order of their distance.
    matches = sorted(matches, key=lambda x: x.distance)
    img3 = cv.drawMatches(img1, kp1, img2, kp2, matches, None, flags=2)

    plt.imshow(img3)
    plt.show()
    
def brute_force(im1, im2):
    """ Brute forse applied on two images. Use SIFT and knnMatcher 
    Returns: the plot with all the matches.
    Example: brute_force('images/im1.jpg','images/im2.jpg')
    """
    img1 = cv.imread(im1, cv.COLOR_BGR2GRAY)  # queryImage
    img2 = cv.imread(im2, cv.COLOR_BGR2GRAY)  # trainImage

    # Initiate SIFT detector
    sift = cv.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    # BFMatcher with default params
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1,des2, k=2)

    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])

    # cv.drawMatchesKnn expects list of lists as matches.
    img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,good,flags=2,outImg=im3)

    plt.imshow(img3),plt.show()

## Tensorflow DELF

Load the neural model.

In [7]:
import tensorflow_hub as hub
delf = hub.load('https://tfhub.dev/google/delf/1').signatures['default']

Functions to extract inliers from images.

In [11]:
from absl import logging

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
from PIL import ImageFile
from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform
from six import BytesIO

import glob , json
import tensorflow as tf


from six.moves.urllib.request import urlopen

def download_and_resize(url, new_width=256, new_height=256):
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    image = Image.open(url)
    image = ImageOps.fit(image, (new_width, new_height), Image.ANTIALIAS)
    return image

def run_delf(image):
    np_image = np.array(image)
    float_image = tf.image.convert_image_dtype(np_image, tf.float32)

    return delf(
        image=float_image,
        score_threshold=tf.constant(100.0),
        image_scales=tf.constant([0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0]),
        max_feature_num=tf.constant(1000))

def match_images(image1, image2, result1, result2):
    distance_threshold = 0.8

    # Perform geometric verification using RANSAC.
    try:
        # Read features.
        num_features_1 = result1['locations'].shape[0]
        num_features_2 = result2['locations'].shape[0]


        # Find nearest-neighbor matches using a KD tree.
        d1_tree = cKDTree(result1['descriptors'])
        _, indices = d1_tree.query(
          result2['descriptors'],
          distance_upper_bound=distance_threshold)

        # Select feature locations for putative matches.
        locations_2_to_use = np.array([
          result2['locations'][i,]
          for i in range(num_features_2)
          if indices[i] != num_features_1
        ])
        locations_1_to_use = np.array([
          result1['locations'][indices[i],]
          for i in range(num_features_2)
          if indices[i] != num_features_1
        ])

        _, inliers = ransac(
          (locations_1_to_use, locations_2_to_use),
          AffineTransform,
          min_samples=3,
          residual_threshold=20,
          max_trials=1000)

        print("Loaded image cat %d features" % num_features_1)
        print("Loaded image user %d features" % num_features_2)
        print('Found %d inliers' % sum(inliers))

        # Visualize correspondences.
        #_, ax = plt.subplots()
        #inlier_idxs = np.nonzero(inliers)[0]
        #plot_matches(ax,image1,image2,locations_1_to_use,locations_2_to_use,np.column_stack((inlier_idxs, inlier_idxs)),matches_color='b')
        #ax.axis('off')
        #title_plot = 'inliers:'+str(sum(inliers))
        #ax.set_title(title_plot)
        return sum(inliers)
    except:
        return 0       

Apply DELF to images in two folders, save results in a JSON.

In [14]:
def delf_(catalogue_folder, user_folder):  
    """
    Match images from two folders using DELF. 
    Returns: a JSON file **for each image in the first folder** (`<image_name>.json`), 
    including the list of matched images ranked by the number of matched inliers (`<image_name> : <int>`).
    Example: delf_("images/dmh/", "images/ig_2000/")
    """
    for catalogue_img in glob.iglob(catalogue_folder+'*'):
        f_name = catalogue_img.split('.jp')[0].split(catalogue_folder)[1]
        match_dict = {}
        match_dict['catalogue_img'] = catalogue_img
        image1 = download_and_resize(catalogue_img)
        try:
            result1 = run_delf(image1)
            for user_file in glob.iglob(user_folder+'*'):     
                image2 = download_and_resize(user_file)
                try:        
                    result2 = run_delf(image2) 
                    img_mtc = match_images(image1, image2, result1, result2)

                    if img_mtc > 0:  
                        match_dict[user_file] = int(img_mtc)

                except Exception as e:
                    match_dict['error'] = 'True'  
        except Exception as e:
            pass 
        with open('delf/delf_'+f_name+'.json', 'w', encoding='utf-8') as outfile:
            json.dump(match_dict, outfile, ensure_ascii=False, indent=1)

In [15]:
delf_("GAM_catalogue_images/", "GAM_instagram_images/")

Loaded image cat 276 features
Loaded image user 155 features
Found 8 inliers
Loaded image cat 276 features
Loaded image user 79 features
Loaded image cat 276 features
Loaded image user 130 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 137 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 120 features
Found 5 inliers
Loaded image cat 276 features
Loaded image user 185 features
Found 6 inliers
Loaded image cat 276 features
Loaded image user 100 features
Found 6 inliers
Loaded image cat 276 features
Loaded image user 153 features
Found 7 inliers
Loaded image cat 276 features
Loaded image user 91 features
Loaded image cat 276 features
Loaded image user 101 features
Found 3 inliers
Loaded image cat 276 features
Loaded image user 70 features
Found 6 inliers
Loaded image cat 276 features
Loaded image user 61 features
Found 5 inliers
Loaded image cat 276 features
Loaded image user 51 features
Found 7 inliers
Loaded image cat 276 features
Loaded

Loaded image cat 276 features
Loaded image user 95 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 135 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 47 features
Loaded image cat 276 features
Loaded image user 165 features
Found 5 inliers
Loaded image cat 276 features
Loaded image user 47 features
Found 3 inliers
Loaded image cat 276 features
Loaded image user 102 features
Loaded image cat 276 features
Loaded image user 111 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 97 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 56 features
Loaded image cat 276 features
Loaded image user 146 features
Found 3 inliers
Loaded image cat 276 features
Loaded image user 70 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 158 features
Found 8 inliers
Loaded image cat 276 features
Loaded image user 85 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 160 fe

Loaded image cat 276 features
Loaded image user 203 features
Found 3 inliers
Loaded image cat 276 features
Loaded image user 107 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 115 features
Found 5 inliers
Loaded image cat 276 features
Loaded image user 98 features
Found 3 inliers
Loaded image cat 276 features
Loaded image user 100 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 57 features
Loaded image cat 276 features
Loaded image user 133 features
Found 8 inliers
Loaded image cat 276 features
Loaded image user 106 features
Found 7 inliers
Loaded image cat 276 features
Loaded image user 123 features
Loaded image cat 276 features
Loaded image user 128 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 96 features
Found 5 inliers
Loaded image cat 276 features
Loaded image user 88 features
Found 7 inliers
Loaded image cat 276 features
Loaded image user 93 features
Found 6 inliers
Loaded image cat 276 features
Loaded

Loaded image cat 276 features
Loaded image user 161 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 122 features
Found 5 inliers
Loaded image cat 276 features
Loaded image user 124 features
Found 6 inliers
Loaded image cat 276 features
Loaded image user 93 features
Found 7 inliers
Loaded image cat 276 features
Loaded image user 47 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 79 features
Found 3 inliers
Loaded image cat 276 features
Loaded image user 56 features
Loaded image cat 276 features
Loaded image user 163 features
Found 6 inliers
Loaded image cat 276 features
Loaded image user 55 features
Found 3 inliers
Loaded image cat 276 features
Loaded image user 59 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 123 features
Found 5 inliers
Loaded image cat 276 features
Loaded image user 99 features
Found 4 inliers
Loaded image cat 276 features
Loaded image user 103 features
Found 3 inliers
Loaded image cat 276 f

Loaded image cat 49 features
Loaded image user 33 features
Loaded image cat 49 features
Loaded image user 106 features
Found 3 inliers
Loaded image cat 49 features
Loaded image user 91 features
Loaded image cat 49 features
Loaded image user 144 features
Loaded image cat 49 features
Loaded image user 96 features
Found 5 inliers
Loaded image cat 49 features
Loaded image user 98 features
Found 5 inliers
Loaded image cat 49 features
Loaded image user 93 features
Found 5 inliers
Loaded image cat 49 features
Loaded image user 71 features
Loaded image cat 49 features
Loaded image user 47 features
Found 4 inliers
Loaded image cat 49 features
Loaded image user 118 features
Found 3 inliers
Loaded image cat 49 features
Loaded image user 133 features
Loaded image cat 49 features
Loaded image user 108 features
Found 8 inliers
Loaded image cat 49 features
Loaded image user 57 features
Found 4 inliers
Loaded image cat 49 features
Loaded image user 201 features
Found 3 inliers
Loaded image cat 49 feat

KeyboardInterrupt: 

Apply FLANN on top of DELF to lower the number of false positives (Spoiler alert, no good results).

In [ ]:
# 
def flann_on_delf(delf_matches_folder):
    """
    Try out flann on the output of images matches with DELF (stored in a folder as JSON file) 
    - to restrict the number of false positives when the number of inliers is high.
    Returns: the plot of the good keypoints matched between images.
    Example: flann_on_delf("images/delf_results/")
    """
    for delf_file in glob.iglob(delf_matches_folder+'*.json'):
        with open(delf_file, 'r', encoding='utf-8') as delf_match:
            data_delf = json.load(delf_match)
            artefact = data_delf['catalogue_img']
            for img_ig, val in data_delf.items():
                if type(val) == int and val > 15:
                    #f, axarr = plt.subplots(2,2)
                    #axarr[0,0].imshow(cv.imread(artefact))
                    #axarr[0,0].set_title(val)
                    #axarr[0,1].imshow(cv.imread(img_ig))

                    flann_(artefact, img_ig)
               

## CNN - Keras

source: [https://towardsdatascience.com/build-an-image-search-engine-using-python-ad181e76441b](https://towardsdatascience.com/build-an-image-search-engine-using-python-ad181e76441b)

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

class FeatureExtractor:
    def __init__(self):
        # Use VGG-16 as the architecture and ImageNet for the weight
        base_model = VGG16(weights='imagenet')
        # Customize the model to return features from fully-connected layer
        self.model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    
    def extract(self, img):
        img = img.resize((224, 224))
        img = img.convert('RGB')
        # Reformat the image
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # Extract Features
        feature = self.model.predict(x)[0]
        return feature / np.linalg.norm(feature)


In [ ]:
fe = FeatureExtractor()

def extract_features_imgs(folder_name):
    """
    Extract features from images stored in "folder_name" using CNN. 
    Returns a numpy array stored in a .npy file for each image.
    Example: features , img_paths = extract_features_imgs("images/ig")
    """
    features = []
    img_paths = []

    for img_path in sorted(Path(folder_name).glob("*")):
            print(img_path)  # e.g., ./static/img/xxx.jpg
            feature = fe.extract(img=Image.open(img_path))
            feature_path = Path("images_feature") / (img_path.stem + ".npy")  # e.g., ./static/feature/xxx.npy
            np.save(feature_path, feature)

    for feature_path in Path("images_feature").glob("*.npy"):
        features.append(np.load(feature_path))
        img_paths.append(Path(folder_name) / (feature_path.stem ))

    features = np.array(features)
    return features , img_paths


In [ ]:
# features , img_paths = extract_features_imgs("images/ig")

In [ ]:
def query_img(img_file, features, img_paths):
    """
    Extract features from a query image "img_file" using CNN 
    and retrieves the best matches among the images previously matched ("features"). 
    Returns a numpy array stored in a .npy file for each image.
    Example: features , img_paths = extract_features_imgs("images/ig")
    """
    img = Image.open(img_file)
    # Extract its features
    query = fe.extract(img)
    dists = np.linalg.norm(features-query, axis=1)  # L2 distances to features
    ids = np.argsort(dists)[:30]  # Top 30 results
    scores = [(dists[id], img_paths[id]) for id in ids]
    return scores 

#### Save results

In [ ]:
from collections import defaultdict
import os

def save_cnn(input_images_folder, features, img_paths, output_json):
    """
    Example: Match all the images in two folders with Keras. 
    Save best matches in a JSON file. 
    features , img_paths = extract_features_imgs("images/ig")
    save_cnn("images/dmh/", features, img_paths, "dmh_ig_images_matching_cnn.json")
    outpt_json : { catalogue_img : [str(matched_img), float(similarity_score)] }
    """
    matches_dict = defaultdict(list)

    for catalogue_img in glob.iglob(input_images_folder+'*'):
        scores = query_img(catalogue_img, features, img_paths)
        for a in range(5*6):
            score = scores[a]
            similarity_score = score[0]
            if similarity_score <= 0.93:
                path = 'images/ig/'
                for i in os.listdir(path):
                    if os.path.isfile(os.path.join(path,i)) and str(score[1])[len(path)+1:] in i:
                        matched_img = path+i
                        matches_dict[catalogue_img].append( [matched_img, str(similarity_score)] )

    matches_dict = dict(matches_dict)

    with open(output_json, 'w', encoding='utf-8') as outfile:
            json.dump(matches_dict, outfile, ensure_ascii=False, indent=1)

In [ ]:
# save_cnn("images/dmh/", features, img_paths, "images/dmh_ig_images_matching_cnn.json")

Plot the 30 best matches for a query image. Results with euclidean distance < .93 seem reliable (very conservative to avoid false positives).

In [ ]:
import os
scores = query_img("images/dmh/wsi-imageoptim-etakortti-22242-320x422.jpg", features, img_paths)

axes=[]
fig=plt.figure(figsize=(8,8))
for a in range(5*6):
    score = scores[a]
    axes.append(fig.add_subplot(5, 6, a+1))
    subplot_title=str(score[0])
    axes[-1].set_title(subplot_title)  
    plt.axis('off')
    path = 'images/ig/'
    for i in os.listdir(path):
        if os.path.isfile(os.path.join(path,i)) and str(score[1])[len(path)+1:] in i:
            plt.imshow(Image.open(path+i))
fig.tight_layout()
plt.show()